In [1]:
import anndata
adata = anndata.read_h5ad('../data/raw/my_data.h5ad')

2025-11-29 15:57:20.571 | INFO     | analysis_src.config:<module>:11 - PROJ_ROOT path is: C:\Users\philu\Philuminati\masked-transformer-comparison


ModuleNotFoundError: No module named 'analysis_src.data'